In [1]:
import pandas as pd
import numpy as np
import spacy
import re
import math
import json
from dataclasses import dataclass, field
from typing import List, Optional

from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

# Transformer libraries
from sentence_transformers import SentenceTransformer # For estimating the distance between (sub)sequences
from sentence_transformers import util

In [2]:
# Instantiate SBERT
sentence_model = SentenceTransformer('all-mpnet-base-v2')

In [2]:
# Load spacy language model
nlp = spacy.load("en_core_web_trf")

# Load the GloVe embeddings
glove_file = datapath('/home/aum/Desktop/projects/nlp/models/glove.6B.100d.txt')
# glove_file = datapath('/home/aumaron/Desktop/nlp/nlp_playground/models/glove.6B/glove.6B.100d.txt')
word2vec_glove_file = get_tmpfile("glove.6B.100d.word2vec.txt")
glove2word2vec(glove_file, word2vec_glove_file)

model = KeyedVectors.load_word2vec_format(word2vec_glove_file)

/tmp/ipykernel_30379/1039543407.py:8: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(glove_file, word2vec_glove_file)


In [3]:
# Dimensional space of the embeddings

model[model.index_to_key[0]].shape[0]

100

In [4]:
entity_data = pd.read_excel("../data/entity_data.xlsx")
root_word_corpus = pd.read_excel("../data/root_word_corpus.xlsx")
column_names = entity_data["column_names"].tolist()
old_entity_corpus = open("../misc/corpus_transformed.json")
old_entity_corpus = json.load(old_entity_corpus)

In [5]:
# column_names

In [5]:
intermediate_corpus = root_word_corpus[['id', 'name', 'entity']].to_dict('records')

In [8]:
column_name = 'branch/name'
column_name =  re.sub(r'[@_!#$%^&*()<>?[\]./\\+|}{~:-]', ' ', column_name)  # Removal of special characters
column_name = re.sub(r"[ ]{2,}", " ", column_name)  # Remove additional spaces
print(column_name)

branch name


In [9]:
# 

query_embedding = sentence_model.encode([column_name])

1. Find NER using Spacy
2. Find individual tokens in intermediate corpus
3. If not found in step 2, find semantically similar words in R^d 100 dimensional space
4. Future scope:
    % is removed as a character
    Certain columns containing '%' in the beginning or end are percentage columns
    Need to add the exception for %
    
5. Challenges -
    - False positives in model based NER
    - False positives in new algorithmic approach
    - Reducing the prediction window (lesser, but more accurate entities should be displayed)
    
6. Future scope and experimentation
    - Implement a heuristic algorithm to funnel out wrong predictions from the predicted universe
        - Syntactic rules can be generalised and implemented
            1. Eg. "Date of user joining" 
                "of" is an adposition (ADP) which we can use to decipher that the prior entity is an important keyword in the entity; which is "Date"
            2. Eg. "units in inventory";
                "Units" -> relates to "Sales";
                "in" -> is the Adposition;
                "inventory" -> relates to "Sales" and "Product";
                But we eliminate Product as we weigh "Units" more in terms of entity
                
    - Use randomly selected values to funnel out entities further
    - Try 200d vectors
    - Try 300d vectors
    - Try 768d BERT embeddings (non context specific word embeddings)
    - Try extracting phrase (in this case column names) embeddings as a whole

In [20]:
# %%timeit

def model_based_ner(string: str) -> list:
    word = nlp(column_name)
    entity_list = []
    for token_num, token in enumerate(word):
#         print(f"{token.text} -> {token.ent_iob_} -> {token.ent_type_}")
        print(f"{token.text} -> {token.pos_}")
        if token.ent_type_:
            entity_list.append(token.ent_type_)
    
    return entity_list

In [14]:
# %%timeit

@dataclass
class CorpusSearch:
    column_name: str
    word_corpus: List[dict]
    word_matrix: np.ndarray
    confidence_required: float
    embedding_model: KeyedVectors
    entity_from_corpus: list = field(default_factory=list)
    
    def __post_init__(self):
        word_split = self.column_name.split(" ")
        for word_num, word in enumerate(word_split):
            self.word_id = word_num
            self.root_word = word
            try:
                self.root_word_embedding = self.embedding_model.get_vector(self.root_word)
            except KeyError:
                self.root_word_embedding = np.zeros((100,))
            self.last_word_embedding = self.root_word_embedding
            self.entity_from_corpus.extend(self.find_in_corpus())
    
    def find_in_corpus(self) -> list:
        entity_list = []
        filtered_list = list(filter(lambda word_meta: word_meta["name"] == self.root_word, 
                                    self.word_corpus))
        meta_obj = {
            "actual_word": self.root_word,
            "entity": [],
            "closest_root:": self.root_word,
            "word_id": self.word_id,
            "confidence": []
        }
        if filtered_list:
            for each_object in filtered_list:
                meta_obj["entity"].append(each_object.get("entity"))
                meta_obj["confidence"].append(1.)
            entity_list.append(meta_obj)
        else:
            closest_index, confidence_value = self.get_closest_word()
            if closest_index or closest_index == 0:
                closest_entity = self.word_corpus[closest_index] if (closest_index or closest_index == 0) else ""
                entity_list = [{"actual_word": self.root_word,
                                "entity": [closest_entity["entity"]],
                                "closest_root": closest_entity["name"],
                                "word_id": self.word_id,
                                "confidence": [confidence_value]}]
            else:
                entity_list = [{"actual_word": self.root_word,
                                "entity": [],
                                "closest_root": "",
                                "word_id": self.word_id,
                                "confidence": [confidence_value]}]

        return entity_list
    
    @staticmethod
    def embedding_product(a, b):
        cos_theta = a.dot(b.T)/(np.sqrt(np.sum(np.square(a)))*(np.sqrt(np.sum(np.square(b)))))
        _angle = math.acos(cos_theta)

        return cos_theta, _angle


    def get_closest_word(self):
        theta_list = []
        angle_list = []
        for column_vec in range(self.word_matrix.T.shape[1]):
            doc_product, angle_between_vectors = self.embedding_product(self.last_word_embedding, 
                                                                        self.word_matrix.T[:, column_vec])
            theta_list.append(doc_product)
            angle_list.append(angle_between_vectors)  # If needed for validation

        # Cut-off: filter theta list based on the confidence required
        filtered_theta_index_list = [
            theta_list.index(score) for score in theta_list if np.abs(score)>=self.confidence_required]

        # Find index of the top score
        closest_vector = None
        theta_value = None
        if filtered_theta_index_list:
            closest_vector = theta_list.index(max([theta_list[filter_index] for filter_index in filtered_theta_index_list]))
            theta_value = theta_list[closest_vector]
#         print("closest_vect",closest_vector)
    #     print(theta_list)
#         print(theta_list.index(max(theta_list)))
    #     print(theta_list[theta_list.index(max(theta_list))])
    #     print(angle_list.index(min(angle_list)))

        return closest_vector, theta_value


In [8]:
# %%timeit

def construct_corpus_matrix(word_corpus: List[dict], embedding_model):
    word_array = np.empty([len(word_corpus), embedding_model[embedding_model.index_to_key[0]].shape[0]])
    for row_number, root in enumerate(word_corpus):
        try:
            word_array[row_number, :] = embedding_model.get_vector(root.get("name"))
        except KeyError:
            word_array[row_number, :] = np.zeros([100,])
        
    return word_array

# corpus_array = construct_corpus_matrix(intermediate_corpus, model)

In [10]:
# %%timeit

In [23]:
# Step 0
corpus_array = construct_corpus_matrix(intermediate_corpus, model)
# print(corpus_array.shape)

entity_from_old_method = []
entity_from_model_list = []
entity_from_search = []
for column_name in ["units in inventory"]:  # entity_data["column_names"].values:
    old_entity_list = [
        word_obj["entity"] for word_obj in list(filter(lambda corpus: corpus.get("name") == column_name, old_entity_corpus))
    ]
#     print("old entities", old_entity_list)
    old_entity_list = list(set(old_entity_list))
    if not old_entity_list:
        old_entity_list = [""]
    entity_from_old_method.append(old_entity_list)
    column_name = column_name.lower()
    column_name =  re.sub(r'[@_!#$%^&*()<>?[\]./\\+|}{~:-]', ' ', column_name)  # Removal of special characters
    column_name = re.sub(r"[ ]{2,}", " ", column_name)  # Remove additional spaces
    # Step 1 
    entity_from_model = model_based_ner(column_name)
    print('Model NER -> ', entity_from_model)
    if entity_from_model:
        entity_from_model = list(set(entity_from_model))
    else:
        entity_from_model = [""]
    entity_from_model_list.append(entity_from_model)

    # Step 2: Find in corpus
    corpus_search_obj = CorpusSearch(
        column_name=column_name,
        word_corpus=intermediate_corpus,
        word_matrix=corpus_array,
        confidence_required=0.5,
        embedding_model=model,
    )

    print('Search in Corpus -> ', corpus_search_obj.entity_from_corpus)

    # Check if no entities found
    non_identified_entity = list(filter(lambda word_is: not word_is.get("entity"), 
                                        corpus_search_obj.entity_from_corpus))
    if non_identified_entity and (len(non_identified_entity) == len(column_name.split(" "))):
        print(f"--------\nNo entities found!\n--------")

    extract_entities = [word_obj["entity"] for word_obj in corpus_search_obj.entity_from_corpus]
    extract_entities = [word for word_list in extract_entities for word in list(set(word_list))]
#     print(extract_entities)
    
    entity_from_search.append(", ".join(extract_entities))
    for i, word in enumerate(entity_from_search):
        if word == "":
            entity_from_search.pop(i)
            entity_from_search.insert(i, "Default")


# entity_from_old_method = [word for word_list in entity_from_old_method for word in word_list]
# entity_from_model_list = [word for word_list in entity_from_model_list for word in word_list]

# entity_data["old_recognition_method"] = entity_from_old_method
# entity_data["model_based_recognition"] = entity_from_model_list
# entity_data["new_algorithm_based_recognition"] = entity_from_search

# if not entity_from_model:
#     word_split = column_name.split(" ")
#     # Step 2.a: Find sub_words in corpus
#     entity_from_corpus = []
#     for word_num, word in enumerate(word_split):
#         entity_from_corpus.extend(find_in_corpus(word, word_num, intermediate_corpus))  # Can be replaced using a mat mul
#     print('Simple search in Corpus -> ', entity_from_corpus)
    
#     # Step 2.b: Find if the last sub_word has returned an entity
#     # Note: There can be 4 possibilities:
#         # 1. All sub words can return entity
#         # 2. Any sub_word other than the trailing sub_word returns an entity
#         # 3. Any sub_word including the trailing sub_word returns an entity
#         # 4. None of them return an entity
    
#     # 2.b.1: Check if all words contain entity
#     non_identified_entity = list(filter(lambda word_is: not word_is.get("entity"), entity_from_corpus))
#     if non_identified_entity and (len(non_identified_entity) == len(word_split)):
#         print(f"--------\nNo entities found!\n--------")
        
#     # 2.b.2:
    
    
#     # 2.b.2: Check if last word has empty entity
#     last_sub_word_entity = list(filter(
#         lambda word_is: (word_is.get("actual_word") == word_split[-1]) and (not word_is.get("entity")), 
#         entity_from_corpus))
#     print('Last word entity -> ', last_sub_word_entity)
    
#     # If non-empty, then check this word in corpus
#     if last_sub_word_entity:
#         theta_list = []
#         word_id = last_sub_word_entity[-1].get("word_id")
#         last_word_embedding = model.get_vector(word_split[-1])
#         closest_index = get_closest_word(last_word_embedding.squeeze()[:100], corpus_array, 0.3)
        
#         if closest_index:
#             closest_entity = intermediate_corpus[closest_index] if (closest_index or closest_index == 0) else ""
#             entity_from_corpus[-1].update({"actual_word": word_split[-1],
#                                            "entity": [closest_entity["entity"]],
#                                            "closest_root": closest_entity["name"],
#                                            "word_id": word_id})
#         else:
#             print(f"--------\nNo entities found!\n--------")
        
#         print('Updated entity object -> ', entity_from_corpus)
    
    


# # corpus_array = construct_corpus_matrix(intermediate_corpus, model)
# # corpus_array.shape

units -> NOUN
in -> ADP
inventory -> NOUN
Model NER ->  []
Search in Corpus ->  [{'actual_word': 'units', 'entity': ['Sales'], 'closest_root': 'supply', 'word_id': 0, 'confidence': [0.5354936262980426]}, {'actual_word': 'in', 'entity': ['Location'], 'closest_root': 'place', 'word_id': 1, 'confidence': [0.7434392096144258]}, {'actual_word': 'inventory', 'entity': ['Sales', 'Product'], 'closest_root:': 'inventory', 'word_id': 2, 'confidence': [1.0, 1.0]}]


/tmp/ipykernel_30379/1396784973.py:84: RuntimeWarning: invalid value encountered in double_scalars
  cos_theta = a.dot(b.T)/(np.sqrt(np.sum(np.square(a)))*(np.sqrt(np.sum(np.square(b)))))


In [17]:
model.similar_by_word("country", topn=50)

[('nation', 0.8758761882781982),
 ('now', 0.7629706263542175),
 ('well', 0.746636688709259),
 ('countries', 0.7459027171134949),
 ('world', 0.7444546818733215),
 ('states', 0.7400078773498535),
 ('has', 0.7369707822799683),
 ('government', 0.73195481300354),
 ('already', 0.7283994555473328),
 ('most', 0.7282326221466064),
 ('part', 0.7277618646621704),
 ('still', 0.7275569438934326),
 ('today', 0.7264338731765747),
 ('united', 0.726360023021698),
 ('all', 0.7252687811851501),
 ('over', 0.7178832292556763),
 ('this', 0.7172205448150635),
 ('europe', 0.7167598009109497),
 ('the', 0.7163448333740234),
 ('people', 0.7160573601722717),
 ('and', 0.7149220108985901),
 ('america', 0.7145408987998962),
 ('since', 0.7144755721092224),
 ('it', 0.7130166888237),
 ('region', 0.7127728462219238),
 ('once', 0.7126500606536865),
 ('one', 0.7116884589195251),
 ('in', 0.7093125581741333),
 ('here', 0.7091266512870789),
 ('but', 0.7043055891990662),
 ('years', 0.7039714455604553),
 ('even', 0.70381140708

In [12]:
entity_data.to_excel("validation_output_final_.xlsx", index=False)

In [30]:
corpus_search_obj.entity_from_corpus

[{'actual_word': 'kk', 'entity': [], 'closest_root': '', 'word_id': 0},
 {'actual_word': 'kk', 'entity': [], 'closest_root': '', 'word_id': 1}]

In [101]:
intermediate_corpus[26]

{'id': 26, 'name': 'company', 'entity': 'Organisation'}

In [19]:
util.cos_sim(model.get_vector("organization"), model.get_vector("organisation"))

tensor([[0.8785]])

In [29]:
sneaker = model.get_vector("man")
shoes = model.get_vector("woman")

dot_prod, angle = vector_cosine(sneaker, shoes)
print(dot_prod, angle)

0.83234936 0.5874632370639816
